## Step 1: Import Required Libraries

In [1]:
!pip install -qU oikan

In [2]:
!pip freeze | grep oikan

oikan==0.0.1.10


In [3]:
import torch
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from oikan.model import OIKAN
from oikan.trainer import train
from oikan.symbolic import extract_symbolic_formula

## Step 2: Load the Feynman equations dataset

In [4]:
# Define 30 - 2 variate equations manually
equations = [
    ("eq1", "np.exp(x1) - np.log(np.abs(x2) + 1e-8)"),  # Exponential minus logarithm
    ("eq2", "np.sqrt(np.abs(x1 * x2) + 1e-8)"),  # Square root of product
    ("eq3", "np.tanh(x1) * np.sinh(x2)"),  # Hyperbolic tangent times hyperbolic sine
    ("eq4", "np.arctan(x1) + np.arcsin(np.clip(x2 / 2, -1, 1))"),  # Inverse trigonometric functions
    ("eq5", "np.log(np.abs(x1) + 1e-8) * np.exp(x2)"),  # Log times exponential
    ("eq6", "np.sinh(x1) + np.cosh(x2)"),  # Sum of hyperbolic functions
    ("eq7", "(x1**2 - x2**2) / (x1 + x2 + 1e-8)"),  # Difference of squares divided
    ("eq8", "np.sqrt(np.abs(x1)) + np.sqrt(np.abs(x2))"),  # Sum of square roots
    ("eq9", "np.tan(x1) * np.tan(x2)"),  # Product of tangents
    ("eq10", "np.power(np.abs(x1) + 1e-8, np.abs(x2))"),  # x1 raised to abs(x2) to prevent NaNs
    ("eq11", "np.exp(-x1**2 - x2**2)"),  # Gaussian-like function
    ("eq12", "np.cos(x1) * np.sin(x2)"),  # Cosine times sine
    ("eq13", "x1 / (np.sqrt(np.abs(x2)) + 1e-8)"),  # x1 divided by sqrt(x2)
    ("eq14", "x1 * np.exp(x2)"),  # Exponential growth with multiplication
    ("eq15", "np.arcsinh(x1) + np.arccosh(np.abs(x2) + 1)"),  # Inverse hyperbolic functions
    ("eq16", "x1**2 / (np.abs(x2) + 1e-8)"),  # Square divided by x2 avoiding negatives
    ("eq17", "np.exp(x1) / (np.exp(x2) + 1e-8)"),  # Exponential ratio
    ("eq18", "np.sinh(x1) * np.cosh(x2)"),  # Product of hyperbolic functions
    ("eq19", "np.tanh(x1 + x2)"),  # Hyperbolic tangent of sum
    ("eq20", "np.sin(x1 * x2)"),  # Sine of product
    ("eq21", "np.exp(x1 + x2)"),  # Exponential of sum
    ("eq22", "x1 * x2 / (np.abs(x1 + x2) + 1e-8)"),  # Product over sum avoiding zero division
    ("eq23", "np.log(np.abs(x1 + x2) + 1e-8)"),  # Log of sum
    ("eq24", "x1 * np.sin(x2)"),  # x1 times sine
    ("eq25", "np.log(np.abs(x1)) + np.log(np.abs(x2) + 1e-8)"),  # Sum of logs
    ("eq26", "np.arctan(x1 / (x2 + 1e-8))"),  # Arctangent ratio
    ("eq27", "np.power(np.abs(x1) + 1e-8, 1 / (np.abs(x2) + 1e-8))"),  # Root operation (avoid NaNs)
    ("eq28", "np.exp(x1) * np.sin(x2)"),  # Exponential times sine
    ("eq29", "np.tan(x1) + np.tan(x2)"),  # Sum of tangents
    ("eq30", "np.sinh(x1) / (np.cosh(x2) + 1e-8)"),  # Hyperbolic ratio
]

## Step 3: Experiment settings

In [5]:
RANDOM_STATE = 42
SAMPLE_SIZE = 1000 

results = pd.DataFrame(columns=['Equation', 'R2 Score', 'MSE', 'MAE', 'Time Taken', 'Original Formula', 'Approximate Formula'])

## Step 4: Approximation of 2 variate equations using OIKAN by training a model on each dataset and extracting symbolic expressions

In [6]:
for name, formula_str in equations:
    try:
        formula = eval(f"lambda x1, x2: {formula_str}")
        
        # Generate synthetic dataset
        X = np.random.uniform(-5, 5, (SAMPLE_SIZE, 2))  # Two input variables
        y = np.array([formula(x1, x2) for x1, x2 in X])
        y = np.nan_to_num(y, nan=0.0, posinf=1e8, neginf=-1e8)  # Handle NaNs and infinities
        y = np.abs(y)  # Ensure non-negative targets for valid R2 score
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
        
        # Convert to tensors
        X_train_torch = torch.FloatTensor(X_train)
        y_train_torch = torch.FloatTensor(y_train).unsqueeze(1)
        X_test_torch = torch.FloatTensor(X_test)
        y_test_torch = torch.FloatTensor(y_test).unsqueeze(1)
        
        # Initialize and train OIKAN model
        model = OIKAN(input_dim=2, output_dim=1, hidden_units=10)
        start_time = time.time()
        train(model, (X_train_torch, y_train_torch), epochs=500, lr=0.005, verbose=False)
        training_time = time.time() - start_time
        
        # Evaluate model
        with torch.no_grad():
            y_pred = np.maximum(model(X_test_torch).numpy().ravel(), 0)  # Ensure predictions are non-negative
        
        r2 = max(0, r2_score(y_test, y_pred))  # Ensure R2 is non-negative
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        
        # Extract symbolic formula
        formula_extracted = extract_symbolic_formula(model, X_test_torch, mode='regression')
    
    except Exception as e:
        print(f"Error processing {name}: {e}")
        r2, mse, mae, training_time, formula_extracted = "INF", "INF", "INF", "INF", "INF"
    
    results.loc[len(results)] = [name, r2, mse, mae, training_time, formula_str, formula_extracted]
    print(f"Processed {name} - R2 Score: {r2}")

Processed eq1 - R2 Score: 0.9974234412665135
Processed eq2 - R2 Score: 0.8100627827503172
Processed eq3 - R2 Score: 0.975034942924532
Processed eq4 - R2 Score: 0.8101523309743432
Processed eq5 - R2 Score: 0.727273098553261
Processed eq6 - R2 Score: 0.992263334414799
Processed eq7 - R2 Score: 0.9689540956790402
Processed eq8 - R2 Score: 0.8740743733095993
Processed eq9 - R2 Score: 0.09487704650053341
Processed eq10 - R2 Score: 0.9811546633658151
Processed eq11 - R2 Score: 0.721558639960565
Processed eq12 - R2 Score: 0.2339662141547575
Processed eq13 - R2 Score: 0.5394494056202517
Processed eq14 - R2 Score: 0.9956406288104611
Processed eq15 - R2 Score: 0.9456968863442996
Processed eq16 - R2 Score: 0.00447532720016397
Processed eq17 - R2 Score: 0.9875878112322178
Processed eq18 - R2 Score: 0.9893768656987081
Processed eq19 - R2 Score: 0
Processed eq20 - R2 Score: 0
Processed eq21 - R2 Score: 0.9735507396085828
Processed eq22 - R2 Score: 0
Processed eq23 - R2 Score: 0.6095557337885822
Proc

## Step 5: Save results


In [7]:
results = results.sort_values(by="R2 Score", ascending=False)
results

,Equation,R2 Score,MSE,MAE,Time Taken,Original Formula,Approximate Formula
0,eq1,0.997423,1.939457e+00,7.898352e-01,10.238453,np.exp(x1) - np.log(np.abs(x2) + 1e-8),(24.33*1) + (-2.54*x(x1)) + (-2.64*x^2(x1)) + ...
13,eq14,0.995641,3.005397e+01,2.635176e+00,5.233925,x1 * np.exp(x2),(-141.61*1) + (101.97*x(x1)) + (-75.26*x^2(x1)...
5,eq6,0.992263,3.656806e+00,1.056669e+00,5.236763,np.sinh(x1) + np.cosh(x2),(1.99*1) + (-7.63*x(x1)) + (7.59*x^2(x1)) + (1...
17,eq18,0.989377,1.800927e+03,2.150269e+01,5.194003,np.sinh(x1) * np.cosh(x2),(6276.18*1) + (304.07*x(x1)) + (17.42*x^2(x1))...
16,eq17,0.987588,1.556229e+04,5.118771e+01,5.268050,np.exp(x1) / (np.exp(x2) + 1e-8),(4046.62*1) + (-369.81*x(x1)) + (631.23*x^2(x1...
9,eq10,0.981155,1.386319e+03,1.500542e+01,5.258565,"np.power(np.abs(x1) + 1e-8, np.abs(x2))",(-3640.75*1) + (-63.51*x(x1)) + (658.88*x^2(x1...
2,eq3,0.975035,8.284451e+00,1.102518e+00,5.907892,np.tanh(x1) * np.sinh(x2),(35.08*1) + (-0.58*x(x1)) + (-1.08*x^2(x1)) + ...
29,eq30,0.974433,1.478502e+00,4.273735e-01,5.374742,np.sinh(x1) / (np.cosh(x2) + 1e-8),(62.08*1) + (0.00*x(x1)) + (11.07*x^2(x1)) + (...
20,eq21,0.973551,1.504326e+04,4.409648e+01,5.357311,np.exp(x1 + x2),(24840.07*1) + (-264.49*x(x1)) + (314.81*x^2(x...
6,eq7,0.968954,1.789254e-01,2.769824e-01,5.098681,(x1**2 - x2**2) / (x1 + x2 + 1e-8),(-3.75*1) + (-5.93*x(x1)) + (2.50*x^2(x1)) + (...


In [8]:
results.to_csv('oikan_2vfa_results.csv', index=False)
print("Results saved to oikan_feynman_results.csv")

Results saved to oikan_feynman_results.csv
